In [6]:
import json

file = json.load(open("en_data_version7.json", "r"))
print(file[1])

{'dialog_id': '000433', 'theme': "cognitive behaviour therapy: exploring inner struggles through imagery, addressing challenges when clients respond with 'i don't know' in therapy.", 'psychotherapy': 'Cognitive Behaviour Therapy', 'topic': 'Job Crisis', 'stage': '', 'guide': '', 'is_same_qa': 0, 'is_same_session': 0, 'background': 'A client struggles with feelings of guilt and inadequacy related to not having a traditional job.', 'reasoning': "Lazarus emphasizes the role of imagery in understanding clients' inner struggles. The step-up technique helps reveal hidden worries, such as feeling trapped and unhappy in a marriage. Therapists may face challenges when clients respond with 'I don't know' despite skilled questioning, requiring suggestions based on case conceptualization or clinical experience to break cognitive blocks.", 'dialog': [{'speaker': 'Supporter', 'participant': 'Therapist', 'content': 'One of your underlying assumptions is ‘If I don’t have a real job then I’m not as goo

In [1]:
!pip uninstall -y unsloth peft

!pip install unsloth trl peft accelerate bitsandbytes

Found existing installation: peft 0.17.0
Uninstalling peft-0.17.0:
  Successfully uninstalled peft-0.17.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.6/47.6 kB 2.5 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.6/307.6 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.9/511.9 kB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 503.9/503.9 kB 36.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 32.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.9/181.9 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 888.1/888.1 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━

In [2]:
import torch
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'None'}")

CUDA available: True
GPU: Tesla T4


In [3]:
from unsloth import FastLanguageModel
import torch

model_name = "unsloth/Phi-3-mini-4k-instruct-bnb-4bit"

max_seq_length = 2048  # Choose sequence length
dtype = None  # Auto detection

# Load model and tokenizer
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=True,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.8.5: Fast Mistral patching. Transformers: 4.55.0.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.26G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/194 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/458 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [7]:
from datasets import Dataset

def format_prompt(example):
    return f"### Input: {example['theme']}\n### Output: {json.dumps(example['dialog'])}<|endoftext|>"

formatted_data = [format_prompt(item) for item in file]
dataset = Dataset.from_dict({"text": formatted_data})

In [8]:
# Add LoRA adapters
model = FastLanguageModel.get_peft_model(
    model,
    r=64,  # LoRA rank - higher = more capacity, more memory
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    lora_alpha=128,  # LoRA scaling factor (usually 2x rank)
    lora_dropout=0,  # Supports any, but = 0 is optimized
    bias="none",     # Supports any, but = "none" is optimized
    use_gradient_checkpointing="unsloth",  # Unsloth's optimized version
    random_state=3407,
    use_rslora=False,  # Rank stabilized LoRA
    loftq_config=None, # LoftQ
)

Unsloth 2025.8.5 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [11]:
from trl import SFTTrainer
from transformers import TrainingArguments

# Training arguments optimized for Unsloth
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    args=TrainingArguments(
        per_device_train_batch_size=1, # Reduced batch size
        gradient_accumulation_steps=8,  # Increased gradient accumulation to compensate
        warmup_steps=10,
        num_train_epochs=3,
        learning_rate=2e-4,
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
        logging_steps=25,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
        save_strategy="epoch",
        save_total_limit=2,
        dataloader_pin_memory=False,
    ),
)

Unsloth: Tokenizing ["text"]:   0%|          | 0/520 [00:00<?, ? examples/s]

In [12]:
# Train the model
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 520 | Num Epochs = 3 | Total steps = 195
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 8 x 1) = 8
 "-____-"     Trainable parameters = 119,537,664 of 3,940,617,216 (3.03% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
25,0.898500
50,0.802600
75,0.751400
100,0.677500
125,0.636900
150,0.566300
175,0.522600


In [20]:
# Test the fine-tuned model
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

# Test prompt
messages = [
    {"Extract an entire conversation "},
]

inputs = tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors="pt",
).to("cuda")

# Generate response
outputs = model.generate(
    input_ids=inputs,
    max_new_tokens=256,
    use_cache=True,
    temperature=0.7,
    do_sample=True,
    top_p=0.9,
)

# Decode and print
response = tokenizer.batch_decode(outputs)[0]
print(response)

<|assistant|> Input: the friendly stomach discussion
### Output: [{"speaker": "Seeker", "participant": "Robert", "content": "Well, I would feel much better. I would be happy.", "id": "513-0", "observation": "Robert expresses a desire for happiness and well-being, indicating a positive emotional state.", "emotional label": ["Happiness"]}, {"speaker": "Supporter", "participant": "Therapist", "content": "What would be the first sign that you would notice?", "id": "513-1", "reasoning": "The therapist prompts Robert to identify specific indicators of his improved emotional state, encouraging self-reflection and awareness.", "strategy": ["Question"]}, {"speaker": "Seeker", "participant": "Robert", "content": "Well, I\u2019d be able to sleep, and I\u2019d wake up, and I\u2019d say, \u201cHey, I


In [26]:
from huggingface_hub import notebook_login

notebook_login()

# Save the model
trainer.save_model("fine_tuned_model")

model.push_to_hub("karlv22/Mental_wellness_model-MWM")
tokenizer.push_to_hub("karlv22/Mental_wellness_model-MWM")

README.md:   0%|          | 0.00/602 [00:00<?, ?B/s]

Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

  ...p94gt2yoa/adapter_model.safetensors:   0%|          | 30.2kB /  478MB            

Saved model to https://huggingface.co/karlv22/Mental_wellness_model-MWM


Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

  /tmp/tmpcux2hotl/tokenizer.model      : 100%|##########|  500kB /  500kB            